In [84]:
from pmdarima import auto_arima
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
NEW_STEPWISE = False

def mase(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    mae_model = np.mean(np.abs(y_true - y_pred))
    mae_naive = np.mean(np.abs(y_true[1:] - y_true[:-1]))
    
    return mae_model / mae_naive if mae_naive != 0 else float('inf')



def printRegressionMetrics(y_true, y_pred):
    
    MAPE = metrics.mean_absolute_percentage_error(y_true, y_pred)
    MSE = metrics.mean_squared_error(y_true, y_pred)
    R2 = metrics.r2_score(y_true, y_pred)
    MASE = mase(y_true, y_pred)
    
    print('MSE:')
    print(MSE)
    print('MAPE:')
    print(MAPE)
    print('MASE:')
    print(MASE)
    print('R2:')
    print(R2)

In [86]:
data = pd.read_csv("./Dataset/dataOld.csv")
data = data.head(80)

In [87]:
data["timestamp"] = data["timestamp"] / 1000
data["timestamp"] = data['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
data.reset_index(drop=True, inplace=True)

In [88]:
if NEW_STEPWISE:
    stepwise_model = auto_arima(teste['price'], trace=True, suppress_warnings=True)
    stepwise_model.summary()

'''

Best model:  ARIMA(4,1,5)(0,0,0)[0]          
Total fit time: 1037.301 seconds

'''

'\n\nBest model:  ARIMA(4,1,5)(0,0,0)[0]          \nTotal fit time: 1037.301 seconds\n\n'

In [89]:
data = data[['timestamp','price']]
data = data.set_index('timestamp')
train,test = train_test_split(teste, test_size=0.3, shuffle=False)
pred = []

for i in range(len(test)):
    model = sm.tsa.ARIMA(train, order=(4,1,5))
    model_fit = model.fit()
    
    pred.append(model_fit.forecast(steps=1)[0])
    
    train = data[:train.shape[0] + i + 1]
    
printRegressionMetrics(test['price'], pred)

/home/mickael/.pyenv/versions/3.11.9/envs/dnnDef/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/mickael/.pyenv/versions/3.11.9/envs/dnnDef/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/mickael/.pyenv/versions/3.11.9/envs/dnnDef/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/tmp/ipykernel_7492/4036376101.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]

MSE:
496.2062288643001
MAPE:
0.002654834148795507
MASE:
3.513815452119737
R2:
-2.539057541470986


/home/mickael/.pyenv/versions/3.11.9/envs/dnnDef/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/tmp/ipykernel_7492/4036376101.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred.append(model_fit.forecast(steps=1)[0])


In [91]:
sns.set_theme(rc={'figure.figsize':(15, 4)})
sns.set_style('whitegrid')
sns.set_context('paper')

ax = data['price'].plot()
ax.set_xlabel("Data")
ax.legend(["Predicted", "Real"])
ax.set_title("Real vs Predicted price chart")

plt.grid(True)

plt.show()

plt.close()

AttributeError: 'Series' object has no attribute 'set_xlabel'